In [39]:
from NSEDownload import stocks
import glob
import pandas as pd
import time
from random import randint
import os

In [40]:
all_n50_scripts = pd.read_csv('data/nifty_50.csv')

This file contains NIFTY50 index as well. Removing it from the list.

In [41]:
all_n50_scripts = list(all_n50_scripts.Symbol)
all_n50_scripts = all_n50_scripts[1:]

In [42]:
completed_files = os.listdir('data\Adjusted')
completed_files = [file.split('.')[0] for file in completed_files if file.endswith('.csv')]

In [43]:
all_n50_scripts = [script for script in all_n50_scripts if script not in completed_files]
print(len(all_n50_scripts))

1


In [44]:
for script in all_n50_scripts:
    try:
        print("Runnning for {}".format(script))
        df = stocks.get_adjusted_stock(stockSymbol = script, start_date = '1-1-2000', end_date = '18-1-2022')
        df = df.reset_index()
        if df.shape[0] > 1:
            df.to_csv("data\Adjusted\{}.csv".format(script), index=False)
    except:
        print("Not successful for {}".format(script))
        time.sleep(randint(3,10))
        pass
    time.sleep(randint(3,10))

Runnning for EICHERMOT
EICHERMOT
Please try again. Error has occured 
 HTTPSConnectionPool(host='www1.nseindia.com', port=443): Max retries exceeded with url: /products/dynaContent/common/productsSymbolMapping.jsp?symbol=EICHERMOT&segmentLink=3&symbolCount%0A%20%0A%0A%0A%0A%0A%0A2%0A&series=EQ&dateRange=+&fromDate=31-12-2000&toDate=30-12-2001&dataType=PRICEVOLUMEDELIVERABLE (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000019DAB33F5B0>, 'Connection to www1.nseindia.com timed out. (connect timeout=20)'))
Please try again. Error has occured 
 HTTPSConnectionPool(host='www1.nseindia.com', port=443): Max retries exceeded with url: /products/dynaContent/common/productsSymbolMapping.jsp?symbol=EICHERMOT&segmentLink=3&symbolCount%0A%20%0A%0A%0A%0A%0A%0A2%0A&series=EQ&dateRange=+&fromDate=27-12-2018&toDate=26-12-2019&dataType=PRICEVOLUMEDELIVERABLE (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000019DAB33F3D0>, 'Connection to 

There are changes in symbol for few companies. Let's replace older symbol with latest one and merge all files into one.

In [86]:
file_path = 'data\Adjusted'
completed_files = os.listdir(file_path)
completed_files = [file for file in completed_files if file.endswith('.csv')]

In [87]:
all_scripts_list =[]
for file in completed_files:
    df = pd.read_csv(file_path + '\\' + file)
    if len(df['Symbol'].unique()) > 1:
        latest_symbol = df.tail(1)['Symbol'].values
        print(latest_symbol)
        df['Symbol'] = latest_symbol[0]
    all_scripts_list.append(df)

['ADANIPORTS']
['AXISBANK']
['BAJFINANCE']
['BHARTIARTL']
['HEROMOTOCO']
['HINDALCO']
['HINDUNILVR']
['INFY']
['JSWSTEEL']
['KOTAKBANK']
['TATACONSUM']
['TATAMOTORS']
['TATASTEEL']
['UPL']


In [88]:
all_scripts_df = pd.concat(all_scripts_list, axis=0, ignore_index=True)

This Data needs some cleaning before consumption. Next few steps will aim towards inspecting every column, changing data type(if required) and cleaning it.

In [95]:
all_scripts_df.Date = pd.to_datetime(all_scripts_df.Date)

In [100]:
all_scripts_df = all_scripts_df.drop(['Series'], axis=1)

In [108]:
all_scripts_df.columns

Index(['Date', 'Symbol', 'Open Price', 'High Price', 'Low Price', 'Last Price',
       'Close Price', 'Average Price', 'Total Traded Quantity', 'Turnover',
       'No. of Trades', 'Deliverable Qty', '% Dly Qt to Traded Qty', ' '],
      dtype='object')

In [110]:
all_scripts_df = all_scripts_df.rename(columns={'Open Price' : 'Open',
'Open Price' : 'Open',
'High Price' : 'High',
'Low Price' : 'Low',
'Last Price' : 'Last',
'Close Price' : 'Close',
'Average Price' : 'Average',
'Total Traded Quantity' : 'Quantity',
'No. of Trades' : 'Trades',
'Deliverable Qty' : 'Deliverable_quatity',
'% Dly Qt to Traded Qty' : 'Deliverable_quatity_pct',
})

In [119]:
all_scripts_df = all_scripts_df[['Date', 'Symbol', 'Open', 'High', 'Low', 'Last', 'Close', 'Average',
       'Quantity', 'Turnover', 'Trades', 'Deliverable_quatity','Deliverable_quatity_pct']]

In [142]:
all_scripts_df.loc[all_scripts_df.Trades=='-','Trades'] = 0
all_scripts_df['Trades'] = all_scripts_df['Trades'].astype('int')

In [160]:
all_scripts_df.loc[all_scripts_df.Deliverable_quatity=='-','Deliverable_quatity'] = -1
all_scripts_df['Deliverable_quatity'] = all_scripts_df['Deliverable_quatity'].astype('int')

In [163]:
all_scripts_df.loc[all_scripts_df.Deliverable_quatity_pct=='-','Deliverable_quatity_pct'] = -1
all_scripts_df['Deliverable_quatity_pct'] = all_scripts_df['Deliverable_quatity_pct'].astype('float64')

In [168]:
all_scripts_df.loc[all_scripts_df.Trades==0, 'Trades'] = (all_scripts_df[all_scripts_df.Trades==0]['Deliverable_quatity'] * 100) / (all_scripts_df[all_scripts_df.Trades==0]['Deliverable_quatity_pct'] )

In [173]:
all_scripts_df = all_scripts_df.sort_values(by=['Symbol','Date'])

In [174]:
all_scripts_df.head()

,Date,Symbol,Open,High,Low,Last,Close,Average,Quantity,Turnover,Trades,Deliverable_quatity,Deliverable_quatity_pct
0,2007-11-27,ADANIPORTS,154.00,210.00,154.0,191.8,192.58,196.944,27294366,2.687719e+10,2.729684e+07,9859619,36.12
1,2007-11-28,ADANIPORTS,196.80,198.00,174.8,177.0,178.78,188.276,4581338,4.312765e+09,4.581583e+06,1453278,31.72
2,2007-11-29,ADANIPORTS,181.80,182.95,168.2,177.4,176.84,177.618,5124121,4.550658e+09,5.122979e+06,1069678,20.88
3,2007-11-30,ADANIPORTS,178.00,191.60,178.0,185.8,184.31,185.834,4609762,4.283257e+09,4.610285e+06,1260913,27.35
4,2007-12-03,ADANIPORTS,187.95,199.00,184.4,196.0,193.86,193.130,2977470,2.875200e+09,2.977464e+06,816123,27.41


In [178]:
all_scripts_df.tail(10)

,Date,Symbol,Open,High,Low,Last,Close,Average,Quantity,Turnover,Trades,Deliverable_quatity,Deliverable_quatity_pct
234798,2022-01-05,WIPRO,720.00,720.85,703.00,713.60,713.50,711.37,7355632,5.232609e+09,135056.0,3418195,46.47
234799,2022-01-06,WIPRO,705.80,712.25,699.95,706.20,705.75,706.11,6441610,4.548514e+09,113645.0,3301841,51.26
234800,2022-01-07,WIPRO,708.00,721.50,707.60,708.10,711.50,714.64,6560163,4.688131e+09,108556.0,2744480,41.84
234801,2022-01-10,WIPRO,709.00,709.00,690.00,694.85,693.50,696.03,13768910,9.583640e+09,299201.0,6950923,50.48
234802,2022-01-11,WIPRO,695.00,698.45,689.00,694.75,694.15,693.99,7743729,5.374055e+09,162859.0,3324495,42.93
234803,2022-01-12,WIPRO,700.00,701.75,683.20,691.00,691.35,689.99,11730341,8.093850e+09,231777.0,4673009,39.84
234804,2022-01-13,WIPRO,663.20,666.00,648.00,650.05,649.75,653.90,30752967,2.010939e+10,549500.0,12454314,40.50
234805,2022-01-14,WIPRO,649.25,649.25,636.50,640.25,639.80,640.71,13257274,8.494094e+09,295017.0,7181140,54.17
234806,2022-01-17,WIPRO,639.50,651.65,633.05,646.20,646.65,644.69,10228938,6.594448e+09,214235.0,4301516,42.05
234807,2022-01-18,WIPRO,648.90,650.00,631.15,634.90,633.30,640.09,7164990,4.586254e+09,208075.0,3788717,52.88


In [181]:
all_scripts_df.head(20)

,Date,Symbol,Open,High,Low,Last,Close,Average,Quantity,Turnover,Trades,Deliverable_quatity,Deliverable_quatity_pct
0,2007-11-27,ADANIPORTS,154.00,210.00,154.00,191.80,192.58,196.944,27294366,2.687719e+10,2.729684e+07,9859619,36.12
1,2007-11-28,ADANIPORTS,196.80,198.00,174.80,177.00,178.78,188.276,4581338,4.312765e+09,4.581583e+06,1453278,31.72
2,2007-11-29,ADANIPORTS,181.80,182.95,168.20,177.40,176.84,177.618,5124121,4.550658e+09,5.122979e+06,1069678,20.88
3,2007-11-30,ADANIPORTS,178.00,191.60,178.00,185.80,184.31,185.834,4609762,4.283257e+09,4.610285e+06,1260913,27.35
4,2007-12-03,ADANIPORTS,187.95,199.00,184.40,196.00,193.86,193.130,2977470,2.875200e+09,2.977464e+06,816123,27.41
5,2007-12-04,ADANIPORTS,197.00,211.20,195.20,209.80,208.29,203.078,4849250,4.923867e+09,4.849155e+06,1537667,31.71
6,2007-12-05,ADANIPORTS,212.20,219.90,210.00,216.80,216.49,216.558,2848209,3.084010e+09,2.848063e+06,904260,31.75
7,2007-12-06,ADANIPORTS,217.80,221.94,210.20,218.02,216.26,217.406,1749516,1.901771e+09,1.749345e+06,825691,47.20
8,2007-12-07,ADANIPORTS,220.00,226.80,215.60,220.00,220.48,221.314,2247904,2.487465e+09,2.247948e+06,697763,31.04
9,2007-12-10,ADANIPORTS,222.00,222.00,212.22,214.71,215.08,216.076,1012350,1.093727e+09,1.012401e+06,417514,41.24


In [182]:
all_scripts_df.head(20).groupby('Symbol')['Close'].shift(-10).rolling(window=9).max()

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
5        NaN
6        NaN
7        NaN
8     231.36
9     239.98
10       NaN
11       NaN
12       NaN
13       NaN
14       NaN
15       NaN
16       NaN
17       NaN
18       NaN
19       NaN
Name: Close, dtype: float64

In [175]:
all_scripts_df['rolling_max_10day'] = (all_scripts_df.groupby('Symbol')['Close']
                                      .apply(lambda x:pd.rolling_max(x, 10)))

AttributeError: module 'pandas' has no attribute 'rolling_max'